In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("src")  # so you can `import grid`, `import solvers`, etc.


In [3]:
print(sys.executable)
print(sys.path[0])


c:\Users\31624\Documents\MIT\Programming\FreqTransfer\.venv\Scripts\python.exe
C:\Program Files\Prodrive Motion Platform\3.80.2.1cb3e041\miscellaneous\pyxml2xml


In [5]:
import os, sys

PROJECT_ROOT = r"C:\Users\31624\Documents\MIT\Programming\FreqTransfer"

# 1) Ensure we run from the project root
os.chdir(PROJECT_ROOT)

# 2) Make sure we import the package (NOT the src folder directly)
#    Remove any existing 'src' path entries or stray Prodrive paths.
sys.path[:] = [
    p for p in sys.path
    if "FreqTransfer\\src" not in p
    and "Prodrive Motion Platform" not in p
]

# 3) Put the project root at the front so `import src` works
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Python:", sys.executable)
print("CWD   :", os.getcwd())
print("sys.path[0:3]:", sys.path[:3])


Python: c:\Users\31624\Documents\MIT\Programming\FreqTransfer\.venv\Scripts\python.exe
CWD   : C:\Users\31624\Documents\MIT\Programming\FreqTransfer
sys.path[0:3]: ['C:\\Users\\31624\\Documents\\MIT\\Programming\\FreqTransfer', 'C:\\Users\\31624\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\31624\\AppData\\Local\\Programs\\Python\\Python312\\DLLs']


In [7]:
from src import (
    GridSpec, FDConfig, PMLConfig, SolverOptions,
    assemble_operator,
    PointSource, RandomPointSource, build_load,
    gmres_solve, plot_residuals, plot_field
)


In [8]:
import os, sys
PROJECT_ROOT = r"C:\Users\31624\Documents\MIT\Programming\FreqTransfer"
os.chdir(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src import (
    GridSpec, FDConfig, PMLConfig, SolverOptions,
    assemble_operator,
    PointSource, RandomPointSource, build_load,
    gmres_solve, plot_residuals, plot_field,
)
